# Pre-processing the data
This notebook will demonstrate transforming the data extracted from MASS into a tabular dataset (alomst) ready for use in machine learning. This notebook will do this for one forecast reference time and one realisation, to make the development process easier (and able to run on a smaller compute instance), and then a separate notebook will be created to do the actual "batch" processing of data.

We are using data around Storm Dennis ([Met Office](https://www.metoffice.gov.uk/weather/warnings-and-advice/uk-storm-centre/storm-dennis), [Wikipedia](https://en.wikipedia.org/wiki/Storm_Dennis))

The key steps in this process are as follows:
* Prepare radar data
  * Load in radar data files extracted from mass and agggregated into 1 file per day of data 
  * Accumuluate in 3hr accumulations to match 3 hour frequency of model data
  * Load a sample MOGREPS-UK UK cutout grid to use a regridding target
  * regrid radar data
  * transform into tabular data
* Prepare MOGREPS-G data
  * Load in data extract from MASS for the forecast reference times and leadtime of interest (IN this case around Storm Dennis 15/16 Feb 2020)
  * Extract UK Data
  * For each forecast ref time,load in all single level variables and transform to tabular (using xarray.Dataset.to_dataframe)
  * For each forecast ref time, load in variables on height levels and tranform to data frame
  * the xarray function by default puts variables on different heights on different rows, whereas we want all levels of a variable for a particular time/lat/lon/realization to be on the same row as separate features/columns. Transform by selecting different heights, renaming variables to include name and height, and merging together.
  * merge single level and height levels variables
  * concatenate different times into a single dataframe and save to disk.

In [1]:
import pathlib
import datetime
import functools
import os

In [2]:
import numpy

In [3]:
import pandas

In [4]:
import xarray
import iris
import iris.quickplot
import iris.coord_categorisation

In [5]:
import matplotlib.pyplot

# Set parameters for notebook
Set the paths and lists of things to process

In [6]:
project_name = 'precip_rediagnosis'
mogreps_g_name = 'mogreps-g'
ilab_project_dir = pathlib.Path('/project/informatics_lab/')
output_dir =  pathlib.Path('/scratch')/ os.environ['USER'] / project_name

In [7]:
root_data_dir = ilab_project_dir / project_name
mogreps_g_data_dir = root_data_dir / mogreps_g_name
radar_data_dir = root_data_dir / 'radar'

In [8]:
output_fname_template = 'prd_{lt:03d}H_{vt.year:04d}{vt.month:02d}{vt.day:02d}T{vt.hour:02d}{vt.minute:02d}Z.csv'

In [9]:
variables_single_level = [
    "cloud_amount_of_total_cloud",
    "rainfall_accumulation-PT03H",
    "snowfall_accumulation-PT03H",
    "rainfall_rate",
    "snowfall_rate",
    "height_of_orography",
    "pressure_at_mean_sea_level",
]

variables_height_levels = [
    "cloud_amount_on_height_levels",
    "pressure_on_height_levels",
    "temperature_on_height_levels",
    "relative_humidity_on_height_levels",
    "wind_direction_on_height_levels",
    "wind_speed_on_height_levels",
    
]

In [10]:
num_periods = 10
start_ref_time = datetime.datetime(2020,2,14,12)
forecast_ref_time_range = [start_ref_time + datetime.timedelta(hours=6)*i1 for i1 in range(num_periods)]
leadtime_hours = 15
realizations_list = list(range(35))

In [11]:
dataset = 'mogreps-g'
subset = 'lev1'
forecast_ref_template = '{frt.year:04d}{frt.month:02d}{frt.day:02d}T{frt.hour:02d}00Z.nc.file'
fname_template = '{vt.year:04d}{vt.month:02d}{vt.day:02d}T{vt.hour:02d}00Z-PT{lead_time:04d}H00M-{var_name}.nc'

In [12]:
variables_to_extract = variables_height_levels + variables_single_level

In [13]:
path_lists_vars = {
    var_name: [f1 for f1 in mogreps_g_data_dir.iterdir() if var_name in str(f1)]
    for var_name in variables_to_extract
}


In [14]:
uk_bounds={'latitude':(50,58), 'longitude': (-6,2)}
xarray_select_uk = {k1: slice(*v1) for k1,v1 in uk_bounds.items()}

## Create a dataset from MOGREPS-G data
Information on Met Office Ensmble forecasts - https://www.metoffice.gov.uk/research/weather/ensemble-forecasting#
Paper - https://www.metoffice.gov.uk/research/weather/ensemble-forecasting 

In [15]:
fcst_ref_time = forecast_ref_time_range[0]
real1 = realizations_list[10]
validity_time = fcst_ref_time + datetime.timedelta(hours=leadtime_hours)

The file names do not match the variables names within the files, so we need to create a mapping to work with.

In [16]:
%%time
# load a cube for each variable in iris to get the actual variable name, and populate dictionary mapping from the var name in the file name to the variable as loaded into iris/xarray
file_to_var_mapping = {
    var_file_name: iris.load_cube(str(mogreps_g_data_dir / fname_template.format(vt=validity_time,
                                                                                 lead_time=leadtime_hours,
                                                                                 var_name=var_file_name))).name()
    for var_file_name in variables_single_level + variables_height_levels}
file_to_var_mapping

CPU times: user 858 ms, sys: 86.2 ms, total: 945 ms
Wall time: 2.24 s


{'cloud_amount_of_total_cloud': 'cloud_area_fraction',
 'rainfall_accumulation-PT03H': 'thickness_of_rainfall_amount',
 'snowfall_accumulation-PT03H': 'lwe_thickness_of_snowfall_amount',
 'rainfall_rate': 'rainfall_rate',
 'snowfall_rate': 'lwe_snowfall_rate',
 'height_of_orography': 'surface_altitude',
 'pressure_at_mean_sea_level': 'air_pressure_at_sea_level',
 'cloud_amount_on_height_levels': 'cloud_volume_fraction_in_atmosphere_layer',
 'pressure_on_height_levels': 'air_pressure',
 'temperature_on_height_levels': 'air_temperature',
 'relative_humidity_on_height_levels': 'relative_humidity',
 'wind_direction_on_height_levels': 'wind_from_direction',
 'wind_speed_on_height_levels': 'wind_speed'}

In [17]:
single_level_var_mappings = {v1: file_to_var_mapping[v1] for v1 in variables_single_level}
height_level_var_mappings = {v1: file_to_var_mapping[v1] for v1 in variables_height_levels}

In [18]:
for fcst_ref_time in forecast_ref_time_range:
    print(fcst_ref_time)

2020-02-14 12:00:00
2020-02-14 18:00:00
2020-02-15 00:00:00
2020-02-15 06:00:00
2020-02-15 12:00:00
2020-02-15 18:00:00
2020-02-16 00:00:00
2020-02-16 06:00:00
2020-02-16 12:00:00
2020-02-16 18:00:00


Create a function to load the data for a particular location (because this project is interested in the UK) and realization (to keep data manageable during development).

In [19]:
def load_realization_ds(ds_path, realization, selected_bounds):
    try:
        subset1 = dict(selected_bounds)
        subset1['realization'] = realization
        subset1['bnds'] = 0
        single_level_ds = xarray.load_dataset(ds_path).sel(**subset1)
    except KeyError as e1:
        single_level_ds = None
    return single_level_ds

In [20]:
%%time
ds_list = [load_realization_ds(
    ds_path= mogreps_g_data_dir / fname_template.format(vt=validity_time,
                                                     lead_time=leadtime_hours,
                                                     var_name=var1),
    realization=real1,
    selected_bounds=xarray_select_uk,
)
    for var1 in variables_single_level]
single_level_ds = xarray.merge([ds1 for ds1 in ds_list if ds1 is not None])


CPU times: user 2.53 s, sys: 340 ms, total: 2.87 s
Wall time: 3.15 s


In [21]:
single_level_ds

<xarray.Dataset>
Dimensions:                           (latitude: 42, longitude: 28)
Coordinates:
    realization                       int32 10
  * latitude                          (latitude) float32 50.16 50.34 ... 57.84
  * longitude                         (longitude) float32 -5.766 ... 1.828
    forecast_period                   timedelta64[ns] 15:00:00
    forecast_reference_time           datetime64[ns] 2020-02-14T12:00:00
    time                              datetime64[ns] 2020-02-15T03:00:00
Data variables:
    cloud_area_fraction               (latitude, longitude) float32 1.0 ... 0...
    latitude_longitude                int32 -2147483647
    latitude_bnds                     (latitude) float32 50.06 50.25 ... 57.75
    longitude_bnds                    (longitude) float32 -5.906 ... 1.688
    thickness_of_rainfall_amount      (latitude, longitude) float32 0.000391 ...
    forecast_period_bnds              int32 43200
    time_bnds                         datetime64[ns] 2020-02-15
    lwe_thickness_of_snowfall_amount  (latitude, longitude) float32 0.0 ... 0.0
    rainfall_rate                     (latitude, longitude) float32 1.863e-08...
    lwe_snowfall_rate                 (latitude, longitude) float32 0.0 ... 0.0
    surface_altitude                  (latitude, longitude) float32 0.0 ... 0.0
    air_pressure_at_sea_level         (latitude, longitude) float32 1.008e+05...
Attributes:
    history:                      2020-02-14T16:23:47Z: StaGE Decoupler
    institution:                  Met Office
    least_significant_digit:      2
    mosg__forecast_run_duration:  PT198H
    mosg__grid_domain:            global
    mosg__grid_type:              standard
    mosg__grid_version:           1.5.0
    mosg__model_configuration:    gl_ens
    source:                       Met Office Unified Model
    title:                        MOGREPS-G Model Forecast on Global 20 km St...
    um_version:                   11.2
    Conventions:                  CF-1.5, UKMO-1.0

First process the single level variables.

In [22]:
%%time
single_level_df = single_level_ds.to_dataframe().reset_index()
single_level_df

CPU times: user 4.92 ms, sys: 2.21 ms, total: 7.13 ms
Wall time: 5.48 ms


,latitude,longitude,cloud_area_fraction,latitude_longitude,realization,latitude_bnds,longitude_bnds,forecast_period,forecast_reference_time,time,thickness_of_rainfall_amount,forecast_period_bnds,time_bnds,lwe_thickness_of_snowfall_amount,rainfall_rate,lwe_snowfall_rate,surface_altitude,air_pressure_at_sea_level
0,50.15625,-5.765625,1.0000,-2147483647,10,50.0625,-5.90625,0 days 15:00:00,2020-02-14 12:00:00,2020-02-15 03:00:00,0.000391,43200,2020-02-15,0.0,1.862645e-08,0.0,0.0000,100829.0
1,50.15625,-5.484375,1.0000,-2147483647,10,50.0625,-5.62500,0 days 15:00:00,2020-02-14 12:00:00,2020-02-15 03:00:00,0.001012,43200,2020-02-15,0.0,3.818423e-08,0.0,31.9375,100878.0
2,50.15625,-5.203125,1.0000,-2147483647,10,50.0625,-5.34375,0 days 15:00:00,2020-02-14 12:00:00,2020-02-15 03:00:00,0.001418,43200,2020-02-15,0.0,5.308539e-08,0.0,45.9375,100921.0
3,50.15625,-4.921875,1.0000,-2147483647,10,50.0625,-5.06250,0 days 15:00:00,2020-02-14 12:00:00,2020-02-15 03:00:00,0.001164,43200,2020-02-15,0.0,3.818423e-08,0.0,0.0000,100972.0
4,50.15625,-4.640625,1.0000,-2147483647,10,50.0625,-4.78125,0 days 15:00:00,2020-02-14 12:00:00,2020-02-15 03:00:00,0.001074,43200,2020-02-15,0.0,3.073364e-08,0.0,0.0000,101022.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1171,57.84375,0.703125,0.0000,-2147483647,10,57.7500,0.56250,0 days 15:00:00,2020-02-14 12:00:00,2020-02-15 03:00:00,0.000000,43200,2020-02-15,0.0,0.000000e+00,0.0,0.0000,100448.0
1172,57.84375,0.984375,0.0000,-2147483647,10,57.7500,0.84375,0 days 15:00:00,2020-02-14 12:00:00,2020-02-15 03:00:00,0.000000,43200,2020-02-15,0.0,0.000000e+00,0.0,0.0000,100469.0
1173,57.84375,1.265625,0.0000,-2147483647,10,57.7500,1.12500,0 days 15:00:00,2020-02-14 12:00:00,2020-02-15 03:00:00,0.000000,43200,2020-02-15,0.0,0.000000e+00,0.0,0.0000,100485.0
1174,57.84375,1.546875,0.1875,-2147483647,10,57.7500,1.40625,0 days 15:00:00,2020-02-14 12:00:00,2020-02-15 03:00:00,0.000000,43200,2020-02-15,0.0,0.000000e+00,0.0,0.0000,100498.0


Load the variables on height levels and create a data frame using the xarray `to_dataframe` function.

In [23]:
%%time
ds_list1 = [load_realization_ds(
    ds_path=mogreps_g_data_dir / fname_template.format(vt=validity_time,
                                                       lead_time=leadtime_hours,
                                                       var_name=var1),
    realization=real1,
    selected_bounds=xarray_select_uk,
)
            for var1 in variables_height_levels]


CPU times: user 2min 18s, sys: 12.1 s, total: 2min 30s
Wall time: 3min 32s


In [24]:
height_levels_ds =  xarray.merge([ds1 for ds1 in ds_list1 if ds1 is not None])

In [25]:
%%time
hl_df_multirow = height_levels_ds.to_dataframe().reset_index()

CPU times: user 10.2 ms, sys: 3.03 ms, total: 13.3 ms
Wall time: 12.5 ms


In [26]:
hl_df_multirow

,height,latitude,longitude,cloud_volume_fraction_in_atmosphere_layer,latitude_longitude,realization,latitude_bnds,longitude_bnds,forecast_period,forecast_reference_time,time,air_pressure,air_temperature,relative_humidity,wind_from_direction,wind_speed
0,5.0,50.15625,-5.765625,0.0,-2147483647,10,50.0625,-5.90625,0 days 15:00:00,2020-02-14 12:00:00,2020-02-15 03:00:00,100769.3750,284.1250,1.004883,200.8125,11.7500
1,5.0,50.15625,-5.484375,0.0,-2147483647,10,50.0625,-5.62500,0 days 15:00:00,2020-02-14 12:00:00,2020-02-15 03:00:00,100436.2500,284.1875,1.007812,203.6250,10.4375
2,5.0,50.15625,-5.203125,0.0,-2147483647,10,50.0625,-5.34375,0 days 15:00:00,2020-02-14 12:00:00,2020-02-15 03:00:00,100312.9375,284.1875,1.001953,206.8750,10.3750
3,5.0,50.15625,-4.921875,0.0,-2147483647,10,50.0625,-5.06250,0 days 15:00:00,2020-02-14 12:00:00,2020-02-15 03:00:00,100912.5000,284.3125,0.977539,207.7500,11.9375
4,5.0,50.15625,-4.640625,0.0,-2147483647,10,50.0625,-4.78125,0 days 15:00:00,2020-02-14 12:00:00,2020-02-15 03:00:00,100962.4375,284.1875,0.977539,208.3750,12.3750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38803,6000.0,57.84375,0.703125,0.0,-2147483647,10,57.7500,0.56250,0 days 15:00:00,2020-02-14 12:00:00,2020-02-15 03:00:00,45534.8750,239.4375,0.164062,264.6250,37.1875
38804,6000.0,57.84375,0.984375,0.0,-2147483647,10,57.7500,0.84375,0 days 15:00:00,2020-02-14 12:00:00,2020-02-15 03:00:00,45545.5000,239.0625,0.198242,262.0625,34.7500
38805,6000.0,57.84375,1.265625,0.0,-2147483647,10,57.7500,1.12500,0 days 15:00:00,2020-02-14 12:00:00,2020-02-15 03:00:00,45557.0625,238.9375,0.234375,258.7500,32.5000
38806,6000.0,57.84375,1.546875,0.0,-2147483647,10,57.7500,1.40625,0 days 15:00:00,2020-02-14 12:00:00,2020-02-15 03:00:00,45569.0000,238.8125,0.276367,255.3750,31.0625


As shown above, the xarray method puts variables at different heights in different rows, where really we want them in the same row in separate columns e.g. `air_temperature_5m`, `air_temperature_10m` etc. So here we select each of the heights in turn and merge the resulting data frames to get what we want.

In [27]:
heights = hl_df_multirow['height'].unique()

In [28]:
print('\n'.join([f'{h1:.2f}m' for h1 in heights]))

5.00m
10.00m
20.00m
30.00m
50.00m
75.00m
100.00m
150.00m
200.00m
250.00m
300.00m
400.00m
500.00m
600.00m
700.00m
800.00m
1000.00m
1250.00m
1500.00m
1750.00m
2000.00m
2250.00m
2500.00m
2750.00m
3000.00m
3250.00m
3500.00m
3750.00m
4000.00m
4500.00m
5000.00m
5500.00m
6000.00m


In [29]:
merge_coords = ['latitude', 'longitude', 'time', 'realization']

In [30]:
coords = list(set(hl_df_multirow.columns) - set(height_level_var_mappings.values()))
print(coords)

['realization', 'forecast_period', 'longitude', 'latitude', 'height', 'latitude_longitude', 'latitude_bnds', 'forecast_reference_time', 'time', 'longitude_bnds']


In [31]:
%%time
var_df_merged = []
# heights_vars_marged = height_levels_df[height_levels_df.height==heights[0]][ merge_coords]
for var1 in height_level_var_mappings.values():
    print(var1)
    # for h1 in heights:
    #     heights_vars_marged[f'{var1}_{h1:.1f}'] = list(height_levels_df[height_levels_df.height==h1][var1])
    var_at_heights = [hl_df_multirow[hl_df_multirow.height==h1][merge_coords + [var1]].rename({var1: f'{var1}_{h1:.1f}'}, axis='columns') for h1 in heights]
    var_df_merged += [functools.reduce(lambda x,y: x.merge(y, on=merge_coords), var_at_heights)]
height_levels_df = functools.reduce(lambda x,y: x.merge(y, on=merge_coords), var_df_merged)

cloud_volume_fraction_in_atmosphere_layer
air_pressure
air_temperature
relative_humidity
wind_from_direction
wind_speed
CPU times: user 1.36 s, sys: 4.18 ms, total: 1.37 s
Wall time: 1.36 s


In [32]:
height_levels_df

,latitude,longitude,time,realization,cloud_volume_fraction_in_atmosphere_layer_5.0,cloud_volume_fraction_in_atmosphere_layer_10.0,cloud_volume_fraction_in_atmosphere_layer_20.0,cloud_volume_fraction_in_atmosphere_layer_30.0,cloud_volume_fraction_in_atmosphere_layer_50.0,cloud_volume_fraction_in_atmosphere_layer_75.0,...,wind_speed_2750.0,wind_speed_3000.0,wind_speed_3250.0,wind_speed_3500.0,wind_speed_3750.0,wind_speed_4000.0,wind_speed_4500.0,wind_speed_5000.0,wind_speed_5500.0,wind_speed_6000.0
0,50.15625,-5.765625,2020-02-15 03:00:00,10,0.0,0.0,0.0,0.0,0.0,0.179688,...,32.1875,31.3125,30.2500,29.6250,29.4375,29.3750,31.3750,38.0000,37.5625,34.6250
1,50.15625,-5.484375,2020-02-15 03:00:00,10,0.0,0.0,0.0,0.0,0.0,0.117188,...,31.2500,31.1250,30.5625,29.8125,29.1875,28.8750,29.5000,36.3750,37.5625,35.6875
2,50.15625,-5.203125,2020-02-15 03:00:00,10,0.0,0.0,0.0,0.0,0.0,0.101562,...,30.1875,30.3750,30.3125,29.7500,28.9375,28.3125,28.8750,34.1875,36.3750,36.0000
3,50.15625,-4.921875,2020-02-15 03:00:00,10,0.0,0.0,0.0,0.0,0.0,0.039062,...,29.3125,29.5000,29.6875,29.3750,28.6875,27.8750,28.6250,32.1250,35.1250,35.8750
4,50.15625,-4.640625,2020-02-15 03:00:00,10,0.0,0.0,0.0,0.0,0.0,0.000000,...,28.7500,28.9375,29.1875,29.0625,28.5625,27.8750,28.4375,31.0000,33.8750,35.3125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1171,57.84375,0.703125,2020-02-15 03:00:00,10,0.0,0.0,0.0,0.0,0.0,0.000000,...,12.6875,12.5000,12.8125,13.6875,15.3750,18.3750,22.0625,25.6875,29.3125,37.1875
1172,57.84375,0.984375,2020-02-15 03:00:00,10,0.0,0.0,0.0,0.0,0.0,0.000000,...,12.8125,12.4375,12.6875,13.4375,15.0625,18.0625,22.6250,25.7500,28.8125,34.7500
1173,57.84375,1.265625,2020-02-15 03:00:00,10,0.0,0.0,0.0,0.0,0.0,0.000000,...,13.3750,12.8125,12.9375,13.5625,14.9375,17.6250,23.0625,25.7500,28.7500,32.5000
1174,57.84375,1.546875,2020-02-15 03:00:00,10,0.0,0.0,0.0,0.0,0.0,0.000000,...,14.5000,13.7500,13.7500,14.2500,15.4375,17.3750,23.4375,25.8750,28.8750,31.0625


Now that we have created the correct dataframe for variables on height levels, we can merge this with the dataframe for single level variables. We are merging on the following coordinates:
* location (latitude and longitude)
* time (validity time)
* realization

In [33]:
mogreps_g_single_ts_uk_df = single_level_df.merge(height_levels_df, on=merge_coords)
mogreps_g_single_ts_uk_df

,latitude,longitude,cloud_area_fraction,latitude_longitude,realization,latitude_bnds,longitude_bnds,forecast_period,forecast_reference_time,time,...,wind_speed_2750.0,wind_speed_3000.0,wind_speed_3250.0,wind_speed_3500.0,wind_speed_3750.0,wind_speed_4000.0,wind_speed_4500.0,wind_speed_5000.0,wind_speed_5500.0,wind_speed_6000.0
0,50.15625,-5.765625,1.0000,-2147483647,10,50.0625,-5.90625,0 days 15:00:00,2020-02-14 12:00:00,2020-02-15 03:00:00,...,32.1875,31.3125,30.2500,29.6250,29.4375,29.3750,31.3750,38.0000,37.5625,34.6250
1,50.15625,-5.484375,1.0000,-2147483647,10,50.0625,-5.62500,0 days 15:00:00,2020-02-14 12:00:00,2020-02-15 03:00:00,...,31.2500,31.1250,30.5625,29.8125,29.1875,28.8750,29.5000,36.3750,37.5625,35.6875
2,50.15625,-5.203125,1.0000,-2147483647,10,50.0625,-5.34375,0 days 15:00:00,2020-02-14 12:00:00,2020-02-15 03:00:00,...,30.1875,30.3750,30.3125,29.7500,28.9375,28.3125,28.8750,34.1875,36.3750,36.0000
3,50.15625,-4.921875,1.0000,-2147483647,10,50.0625,-5.06250,0 days 15:00:00,2020-02-14 12:00:00,2020-02-15 03:00:00,...,29.3125,29.5000,29.6875,29.3750,28.6875,27.8750,28.6250,32.1250,35.1250,35.8750
4,50.15625,-4.640625,1.0000,-2147483647,10,50.0625,-4.78125,0 days 15:00:00,2020-02-14 12:00:00,2020-02-15 03:00:00,...,28.7500,28.9375,29.1875,29.0625,28.5625,27.8750,28.4375,31.0000,33.8750,35.3125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1171,57.84375,0.703125,0.0000,-2147483647,10,57.7500,0.56250,0 days 15:00:00,2020-02-14 12:00:00,2020-02-15 03:00:00,...,12.6875,12.5000,12.8125,13.6875,15.3750,18.3750,22.0625,25.6875,29.3125,37.1875
1172,57.84375,0.984375,0.0000,-2147483647,10,57.7500,0.84375,0 days 15:00:00,2020-02-14 12:00:00,2020-02-15 03:00:00,...,12.8125,12.4375,12.6875,13.4375,15.0625,18.0625,22.6250,25.7500,28.8125,34.7500
1173,57.84375,1.265625,0.0000,-2147483647,10,57.7500,1.12500,0 days 15:00:00,2020-02-14 12:00:00,2020-02-15 03:00:00,...,13.3750,12.8125,12.9375,13.5625,14.9375,17.6250,23.0625,25.7500,28.7500,32.5000
1174,57.84375,1.546875,0.1875,-2147483647,10,57.7500,1.40625,0 days 15:00:00,2020-02-14 12:00:00,2020-02-15 03:00:00,...,14.5000,13.7500,13.7500,14.2500,15.4375,17.3750,23.4375,25.8750,28.8750,31.0625


In [34]:
prd_single_timestep_df = single_level_df.merge(height_levels_df, on=merge_coords)

In [35]:
prd_single_timestep_df

,latitude,longitude,cloud_area_fraction,latitude_longitude,realization,latitude_bnds,longitude_bnds,forecast_period,forecast_reference_time,time,...,wind_speed_2750.0,wind_speed_3000.0,wind_speed_3250.0,wind_speed_3500.0,wind_speed_3750.0,wind_speed_4000.0,wind_speed_4500.0,wind_speed_5000.0,wind_speed_5500.0,wind_speed_6000.0
0,50.15625,-5.765625,1.0000,-2147483647,10,50.0625,-5.90625,0 days 15:00:00,2020-02-14 12:00:00,2020-02-15 03:00:00,...,32.1875,31.3125,30.2500,29.6250,29.4375,29.3750,31.3750,38.0000,37.5625,34.6250
1,50.15625,-5.484375,1.0000,-2147483647,10,50.0625,-5.62500,0 days 15:00:00,2020-02-14 12:00:00,2020-02-15 03:00:00,...,31.2500,31.1250,30.5625,29.8125,29.1875,28.8750,29.5000,36.3750,37.5625,35.6875
2,50.15625,-5.203125,1.0000,-2147483647,10,50.0625,-5.34375,0 days 15:00:00,2020-02-14 12:00:00,2020-02-15 03:00:00,...,30.1875,30.3750,30.3125,29.7500,28.9375,28.3125,28.8750,34.1875,36.3750,36.0000
3,50.15625,-4.921875,1.0000,-2147483647,10,50.0625,-5.06250,0 days 15:00:00,2020-02-14 12:00:00,2020-02-15 03:00:00,...,29.3125,29.5000,29.6875,29.3750,28.6875,27.8750,28.6250,32.1250,35.1250,35.8750
4,50.15625,-4.640625,1.0000,-2147483647,10,50.0625,-4.78125,0 days 15:00:00,2020-02-14 12:00:00,2020-02-15 03:00:00,...,28.7500,28.9375,29.1875,29.0625,28.5625,27.8750,28.4375,31.0000,33.8750,35.3125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1171,57.84375,0.703125,0.0000,-2147483647,10,57.7500,0.56250,0 days 15:00:00,2020-02-14 12:00:00,2020-02-15 03:00:00,...,12.6875,12.5000,12.8125,13.6875,15.3750,18.3750,22.0625,25.6875,29.3125,37.1875
1172,57.84375,0.984375,0.0000,-2147483647,10,57.7500,0.84375,0 days 15:00:00,2020-02-14 12:00:00,2020-02-15 03:00:00,...,12.8125,12.4375,12.6875,13.4375,15.0625,18.0625,22.6250,25.7500,28.8125,34.7500
1173,57.84375,1.265625,0.0000,-2147483647,10,57.7500,1.12500,0 days 15:00:00,2020-02-14 12:00:00,2020-02-15 03:00:00,...,13.3750,12.8125,12.9375,13.5625,14.9375,17.6250,23.0625,25.7500,28.7500,32.5000
1174,57.84375,1.546875,0.1875,-2147483647,10,57.7500,1.40625,0 days 15:00:00,2020-02-14 12:00:00,2020-02-15 03:00:00,...,14.5000,13.7500,13.7500,14.2500,15.4375,17.3750,23.4375,25.8750,28.8750,31.0625


In [36]:
prd_single_timestep_df['time'].min(), prd_single_timestep_df['time'].max()

(Timestamp('2020-02-15 03:00:00'), Timestamp('2020-02-15 03:00:00'))

### Load radar data

Now we want to load the radar, to add a radar rainfall measurement to each column.

In [37]:
radar_days = [datetime.datetime(2020,2,14) + datetime.timedelta(days=d1) for d1 in range(5)]
radar_days

[datetime.datetime(2020, 2, 14, 0, 0),
 datetime.datetime(2020, 2, 15, 0, 0),
 datetime.datetime(2020, 2, 16, 0, 0),
 datetime.datetime(2020, 2, 17, 0, 0),
 datetime.datetime(2020, 2, 18, 0, 0)]

In [38]:
radar_fname_template = 'composite_rainfall_{dt.year:04d}{dt.month:02d}{dt.day:02d}.nc'

Create a single cube of all the radar for the period concerned that has been extract from MASS.
(See `extract_mass_radar.py` for details.)

In [39]:
radar_cube = iris.cube.CubeList([iris.load_cube(str(radar_data_dir / radar_fname_template.format(dt=dt1))) for dt1 in radar_days] ).concatenate_cube()

In [40]:
radar_cube

Rainfall Rate Composite (mm/h),time,projection_y_coordinate,projection_x_coordinate
Shape,1440,2175,1725
Dimension coordinates,,,
time,x,-,-
projection_y_coordinate,-,x,-
projection_x_coordinate,-,-,x
Auxiliary coordinates,,,
forecast_reference_time,x,-,-
Scalar coordinates,,,forecast_period 0 second
Attributes,,,Conventions CF-1.7field_code 213institution Met Officenimrod_version 2probability_period_of_event 0source Plr single site radarstitle Unknown


In [41]:
min([datetime.datetime(c1.point.year, c1.point.month, c1.point.day, c1.point.hour, c1.point.minute) for c1 in radar_cube.coord('time').cells()])

datetime.datetime(2020, 2, 14, 0, 0)

Radar data is instantaeous rainfall rates, measured every 5 minutes. Model data is every three hours. TO match these together, we will calculate "pseudo-accumulations" (pseudo because we're assuming that the instaneous rate represents 5 minute accumulations if we divide by 12, but the rain rate will not be constant in a 5 minute period.) Something we could consider would be some better statistical model to interpolate and do better accumulation calculations, but this is a starting point.

In [42]:
iris.coord_categorisation.add_hour(radar_cube, coord='time')
iris.coord_categorisation.add_day_of_year(radar_cube, coord='time')

In [43]:
coord_3hr = iris.coords.AuxCoord(radar_cube.coord('hour').points // 3,
                                long_name='3hr',
                                 units='hour',
                                )
radar_cube.add_aux_coord(coord_3hr, data_dims=0)

In [44]:
radar_agg_3hr = radar_cube.aggregated_by(['3hr', 'day_of_year'],iris.analysis.SUM)
radar_agg_3hr

Rainfall Rate Composite (mm/h),time,projection_y_coordinate,projection_x_coordinate
Shape,40,2175,1725
Dimension coordinates,,,
time,x,-,-
projection_y_coordinate,-,x,-
projection_x_coordinate,-,-,x
Auxiliary coordinates,,,
3hr,x,-,-
day_of_year,x,-,-
forecast_reference_time,x,-,-
hour,x,-,-


In [45]:
mogreps_g_example = iris.load_cube(
    str(mogreps_g_data_dir / fname_template.format(
        vt=forecast_ref_time_range[0] + datetime.timedelta(hours=leadtime_hours), 
        lead_time=leadtime_hours, 
        var_name=variables_single_level[0])),
    iris.Constraint(latitude=lambda cell1: uk_bounds['latitude'][0] < cell1 < uk_bounds['latitude'][1], 
                                                     longitude=lambda cell1: uk_bounds['longitude'][0] < cell1 < uk_bounds['longitude'][1], realization=0)
)


In [ ]:
# radar_agg_3hr.extract(
#         iris.Constraint(latitude=lambda cell1: mogreps_g_example.coord('latitude').bounds[0][0] < cell1 <mogreps_g_example.coord('latitude').bounds[0][1], 
#                         longitude=lambda cell1: mogreps_g_example.coord('longitude').bounds[0][0] < cell1 <mogreps_g_example.coord('longitude').bounds[0][1])
# )

In [48]:
radar_agg_3hr

Rainfall Rate Composite (mm/h),time,projection_y_coordinate,projection_x_coordinate
Shape,40,2175,1725
Dimension coordinates,,,
time,x,-,-
projection_y_coordinate,-,x,-
projection_x_coordinate,-,-,x
Auxiliary coordinates,,,
3hr,x,-,-
day_of_year,x,-,-
forecast_reference_time,x,-,-
hour,x,-,-


In [49]:
radar_crs = radar_cube.coord_system().as_cartopy_crs()

/home/h01/shaddad/.conda/envs/prd_data_prep/lib/python3.8/site-packages/iris/coord_systems.py:525: UserWarning: The default value for the *approx* keyword argument to TransverseMercator will change from True to False after 0.18.
  return ccrs.TransverseMercator(


In [50]:
proj_y_grid = numpy.tile(radar_cube.coord('projection_y_coordinate').points.reshape(radar_cube.shape[1],1), [1, radar_cube.shape[2]])
proj_x_grid = numpy.tile(radar_cube.coord('projection_x_coordinate').points.reshape(1,radar_cube.shape[2]), [ radar_cube.shape[1],1])

In [51]:
ret_val = mogreps_g_example.coord_system().as_cartopy_crs().transform_points(
    radar_crs,
    proj_y_grid,
    proj_x_grid,
    )


In [52]:
lat_vals = ret_val[:,:,1]
lon_vals = ret_val[:,:,0]

In [53]:
lon_coord = iris.coords.AuxCoord(
    lon_vals,
    standard_name='longitude',
    units='degrees',
)
lat_coord = iris.coords.AuxCoord(
    lat_vals,
    standard_name='latitude',
    units='degrees',
)

In [54]:
radar_agg_3hr.add_aux_coord(lon_coord,[1,2])

In [55]:
radar_agg_3hr.add_aux_coord(lat_coord,[1,2])

In [56]:
radar_agg_3hr

Rainfall Rate Composite (mm/h),time,projection_y_coordinate,projection_x_coordinate
Shape,40,2175,1725
Dimension coordinates,,,
time,x,-,-
projection_y_coordinate,-,x,-
projection_x_coordinate,-,-,x
Auxiliary coordinates,,,
3hr,x,-,-
day_of_year,x,-,-
forecast_reference_time,x,-,-
hour,x,-,-


In [57]:
lat_mog_g_index  = numpy.zeros((radar_cube.shape[1],radar_cube.shape[2]))
lon_mog_g_index  = numpy.zeros((radar_cube.shape[1],radar_cube.shape[2]))

In [59]:
%%time
for i_lon, bnd_lon in enumerate(mogreps_g_example.coord('longitude').bounds):
    print(i_lon)
    for i_lat, bnd_lat in enumerate(mogreps_g_example.coord('latitude').bounds):
        arr1, arr2 = numpy.where((lat_vals >= bnd_lat[0]) &
         (lat_vals < bnd_lat[1])&
         (lon_vals>= bnd_lon[0]) &
         (lon_vals < bnd_lon[1])
        )
        lon_mog_g_index[arr1, arr2] = i_lon
        lat_mog_g_index[arr1, arr2] = i_lat
        
        

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
CPU times: user 51 s, sys: 6.71 ms, total: 51 s
Wall time: 51 s


In [60]:
def compare_time(t1, t2):
    return (t1.year == t2.year) and  (t1.month == t2.month) and  (t1.day == t2.day) and  (t1.hour== t2.hour) and  (t1.minute == t2.minute) 

In [61]:
select_time = [c1.point for c1 in mogreps_g_example.coord('time').cells()][0]

In [62]:
select_time

cftime.DatetimeGregorian(2020, 2, 15, 3, 0, 0, 0, has_year_zero=False)

In [63]:
radar_select_time = radar_agg_3hr.extract(iris.Constraint(time=lambda c1: compare_time(c1.bound[0], select_time )))
radar_select_time 

<iris 'Cube' of Rainfall rate Composite / (mm/h) (projection_y_coordinate: 2175; projection_x_coordinate: 1725)>

In [67]:
improver_thresholds = {
"0.0": [0.0, 0.027],
"0.03": [0.027, 0.033],
"0.09": [0.033, 0.099],
"0.1": [0.099, 0.11],
"0.25": [0.11, 0.275],
"0.3": [0.275, 0.33],
"0.5": [0.33, 0.55],
"1.0": [0.55, 1.1],
"2.0": [1.1, 2.2],
"3.0": [2.2, 3.3],
"4.0": [3.3, 4.4],
"8.0": [4.4, 8.8],
"12.0": [8.8, 13.2],
"16.0": [13.2, 17.6],
"20.0": [17.6, 22.0],
"25.0": [22.0, 27.5],
"30.0": [27.5, 33.0],
"40.0": [33.0, 44.0],
"50.0": [44.0, 55.0],
"75.0": [55.0, 82.5],
"100.0": [82.5, 110.0],
"150.0": [110.0, 165.0],
"200.0": [165.0, 220.0]
}


In [97]:
bands_data = numpy.zeros([mogreps_g_example.shape[0], mogreps_g_example.shape[1], len(improver_thresholds)])

In [98]:
max_rain_data = numpy.zeros([mogreps_g_example.shape[0], mogreps_g_example.shape[1]])

In [99]:
radar_data1 = radar_select_time.data.data

In [ ]:
%%time
for i_lat in range(mogreps_g_example.shape[0]):
    print(i_lat)
    for i_lon in range(mogreps_g_example.shape[1]):
        selected_cells = (~(radar_agg_3hr[0,:,:].data.mask)) & (lat_mog_g_index == i_lat)  & (lon_mog_g_index ==i_lon)
        radar_cells_in_mg = numpy.count_nonzero(selected_cells)
        for imp_ix, (imp_key, imp_bounds) in enumerate(improver_thresholds.items()):
            
            num_in_band = numpy.count_nonzero((radar_data1 >=  imp_bounds[0]) & (radar_data1 <= imp_bounds[1]) & selected_cells)
            bands_data[i_lat, i_lon, imp_ix] = num_in_band / radar_cells_in_mg
            
        # calculate the max radar cell within each mogreps-g cell
        max_rain_data[i_lat, i_lon] = numpy.ma.MaskedArray(
            radar_data1,
            ~selected_cells).max()

0
1
2
3
4


In [ ]:
mg_lat_coord = mogreps_g_example.coord('latitude')
mg_lon_coord = mogreps_g_example.coord('longitude')
radar_time = radar_agg_3hr.coord('time')

In [ ]:
band_coord = iris.coords.DimCoord(
    [float(b1) for b1 in improver_thresholds.keys()],
    bounds=list(improver_thresholds.values())
    var_name='band',
    units='degrees',
)

In [ ]:
fraction_rain_band = iris.cube.Cube(
    data=bands_data, 
    dim_coords_and_dims=((mg_lat_coord, 0),(mg_lon_coord, 1), (band_coord, 2)),
    units=None,
    var_name='fraction_in_band',
    long_name='Fraction radar rainfall cells in specified rain band',
)

In [ ]:
min(mogreps_g_example.coord('longitude').points), max(mogreps_g_example.coord('longitude').points), min(mogreps_g_example.coord('latitude').points), max(mogreps_g_example.coord('latitude').points)

In [ ]:
(min(radar_agg_3hr.coord('longitude').points.flatten()), max(radar_agg_3hr.coord('longitude').points.flatten()), min(radar_agg_3hr.coord('latitude').points.flatten()), max(radar_agg_3hr.coord('latitude').points.flatten()))

In [ ]:
lat_mog_g_index[arr1, arr2] = 12

In [ ]:
lat_mog_g_index.sum()/12

In [ ]:
for bnd_lon in mogreps_g_example.coord('longitude').bounds:
    for bnd_lat in mogreps_g_example.coord('latitude').bounds:
        print(f'lat from {bnd_lat[0]} to {bnd_lat[1]}; long from {bnd_lon[0]} to {bnd_lon[1]}')
        radar_agg_3hr.extr


When we aggregate the data into 3 hour accumulations, for each hour each sum 12 observations, each of of which are giving us a rainfall rate in mm per hour. Our assumption is that represents the rainfall rate for a 5 minute period (60 minutes divided by 12 observations), so we want to translate this inot how many mm will fall in that 5 minute period. So we need to scale the value by 1/12 to get a 5 minute accumulation, which when aggregate all the 5 minute periods will give us correct 3 hour accumulations.

In [ ]:
radar_agg.data = radar_agg.data * (1.0 /12.0)

In [ ]:
radar_agg.add_aux_coord(iris.coords.AuxCoord([c1.bound[0] + datetime.timedelta(hours=3) for c1 in radar_agg.coord('time').cells()], long_name='model_accum_time', units='mm/h'), data_dims=0)

In [ ]:
improver_thresholds['1.0']

In [ ]:
((radar_agg.data.data > improver_thresholds['1.0'][0]) & (radar_agg.data.data < improver_thresholds['1.0'][1]))[10,:,:].sum()

In [ ]:
iris.quickplot.contourf(radar_agg [12,:,:])
matplotlib.pyplot.gca().coastlines()

Load a sample variable from MOGREPS-G to use for regridding radar data.

In [ ]:
mogreps_g_example

In [ ]:
mogreps_g_example.coord('latitude').points

In [ ]:
mogreps_g_example.coord('longitude').bounds[0], mogreps_g_example.coord('latitude').bounds[0]

In [ ]:
rain_bands_cube = iris.cube.copy.deepcopy(radar_agg)

In [ ]:
help(rain_bands_cube.add_dim_coord)

In [ ]:
rain_bands_cube.add_dim_coord(
    iris.coords.DimCoord(
        points=[float(v1) for v1 in improver_thresholds.keys()], 
        bounds=list(improver_thresholds.values())))

In [ ]:
radar_agg.add_dim_coord

In [ ]:
radar_mggrid = radar_agg.regrid(mogreps_g_example, iris.analysis.Linear())

In [ ]:
radar_mggrid

In [ ]:
iris.quickplot.contourf(radar_mggrid [14,:,:])
matplotlib.pyplot.gca().coastlines()

It is slightly tortuous, but having used the useful Iris functionality, we now swiotch to xarray to use the `to_dataframe` functionality.We could probably stick to one or the other rather, and that is a potential future refactoring (if possible).

In [ ]:
radar_df = xarray.DataArray.from_iris(radar_mggrid).to_dataframe().reset_index()

In [ ]:
radar_df

In [ ]:
radar_df = radar_df.rename({'time': 'period_midpoint', : 'time'}, axis='columns')
radar_df['time'] = radar_df['model_accum_time'].apply(lambda dt1: datetime.datetime(dt1.year, dt1.month, dt1.day, dt1.hour, dt1.minute))
radar_df

In [ ]:
prd_single_timestep_df['time'][0]

In [ ]:
radar_df = radar_df[['latitude', 'longitude','time','rainfall_rate_composite']]

In [ ]:
single_ts_ds = xarray.merge([single_level_ds, height_levels_ds])

Now we have the radar data, we can merge with model data and save to disk.

In [ ]:
prd_single_timestep_df.merge(radar_df[radar_df['time'] == prd_single_timestep_df['time'][0]], on=['latitude', 'longitude', 'time'])

# Output as daily parquet files

In [ ]:
output_fname = output_fname_template.format(lt=leadtime_hours, vt=validity_time)
output_path = output_dir / output_fname
prd_single_timestep_df.to_csv(output_path)

Currently I don't have parquet set up in our environments, so just using CSV for now (this is a small data set so far).